## Часть 1. Пример проблемы использования метода Гаусса для решения СЛАУ

Перед вами простая реализация метода Гаусса для решения СЛАУ. Далее по коду представлены две СЛАУ $A_1x = b_1$ и $A_2x = b_2$, эквивалентные с точностью до перестановки строк. Эти СЛАУ решаются сначала пакетным методом, затем реализованным методом Гаусса. Для пакетного метода получается два одинаковых решения. Для метода Гаусса - два отличающихся решения. ЗАДАНИЕ: необходимо объяснить, почему для представленного метода Гаусса решения различаются.

In [628]:
import numpy as np
import numpy.linalg as la
import copy 

def gauss(A_in, b_in):
    n = b_in.size
    A = copy.deepcopy(A_in)
    b = copy.deepcopy(b_in)

    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if A[i,k] != 0:
                c = A[i,k]/A[k,k]
                A[i,(k + 1):n] = A[i,(k + 1):n] - c * A[k,(k + 1):n]
                b[i] = b[i] - c * b[k]
    # Обратный ход
    for k in range(n - 1, -1, -1):
        b[k] = (b[k] - np.dot(A[k,(k + 1):n], b[(k + 1):n]))/A[k,k]

    return b

In [629]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]])
b1 = np.array([0., 0., 1.])

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

mu1 =  16.39373162228438
mu2 =  16.393731622284395


In [630]:
print('u1 = ', la.solve(A1, b1))
print('u2 = ',la.solve(A2, b2))

u1 =  [1. 1. 1.]
u2 =  [1. 1. 1.]


In [631]:
print('u1 = ', gauss(A1, b1))
print('u2 = ', gauss(A2, b2))

u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


**Вследствие невыполнения условия диагонального преобладания и ошибки округления** для представленного метода Гаусса значения различаются. В системе $A_1 u_1 = b_1$ деление на малое значение происходит на первом шаге, что приводит к накоплению ошибок округления. В системе $A_2 u_2 = b_2$ малый элемент стоит на другой позиции и накопление ошибок происходит по-другому.

Таким образом, хотя системы эквивалентны математически, в численном методе порядок операций приводит к различным ошибкам округления, что и вызывает расхождение решений.

## Часть 2. LU разложение

Задание: 

Реализовать алгоритм решения предыдущей задачи с матрицей $A_2$ с помощью $LU$-разложение. В решении должна выводиться $L$, $U$ и решение системы. 

ВАЖНО: реализация метода $LU$ должна быть получена путем небольшой модификации метода $\textit{gauss}$!  При этом саму реализацию можно разделить на два метода: один метод собственно находит $LU$ разложение (можно сделать переделкой цикла для матрицы $A$ метода $\textit{gauss}$), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя пользоваться пакетными методами (в частности, la.solve)

### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [632]:
import sympy as sp

A = sp.Matrix([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
L, U, _ = A.LUdecomposition()
L

Matrix([
[      1,                 0, 0],
[   -0.5,                 1, 0],
[5.0e-17, 0.666666666666667, 1]])

In [633]:
U

Matrix([
[2.0, -1.0,                0.0],
[  0,  1.5,               -1.0],
[  0,    0, -0.333333333333333]])

In [634]:
def lu_decomposition(A_in):
    n = A_in.shape[0]
    A = copy.deepcopy(A_in)
    
    L = np.eye(n)
    
    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if A[i,k] != 0:
                # Заполняем столбцы матрицы L
                L[i,k] = A[i,k] / A[k,k]

                A[i,k:n] = A[i,k:n] - L[i,k] * A[k,k:n]

    U = np.triu(A)

    return L, U

def lu_solve(L, U, b):
    n = b.size

    # Решаем Lv = b (ПРЯМОЙ ХОД)
    v = np.zeros(n)
    for i in range(0, n):
        v[i] = b[i] - np.dot(L[i, 0:i], v[0:i])
    
    # Решаем Uu = v (ОБРАТНЫЙ ХОД)
    u = np.zeros(n)
    for k in range(n - 1, -1, -1):
        u[k] = (v[k] - np.dot(U[k,(k + 1):n], u[(k + 1):n])) / U[k,k]
    
    return u

L, U = lu_decomposition(A2)

print("L:")
display(sp.Matrix(L))

print("U:")
display(sp.Matrix(U))

print("Решение системы:")
display(sp.Matrix(lu_solve(L, U, b2)))

L:


Matrix([
[    1.0,               0.0, 0.0],
[   -0.5,               1.0, 0.0],
[5.0e-17, 0.666666666666667, 1.0]])

U:


Matrix([
[2.0, -1.0,                0.0],
[0.0,  1.5,               -1.0],
[0.0,  0.0, -0.333333333333333]])

Решение системы:


Matrix([
[1.0],
[1.0],
[1.0]])

## Часть 3. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием $LU$-разложения и найти обратную матрицу с точностью $\varepsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.linalg.inv.

In [635]:
A = np.array([[1., 1., 1.], [0., 1., 2.], [7., 1., 4.]])

def lu_inv(A_in):
    n = A_in.shape[0]

    L, U = lu_decomposition(A_in)
    
    E     = np.eye(n)
    A_inv = np.zeros((n, n))
    
    for i in range(0, n):
        e = E[0:n, i]
        A_inv[:, i] = lu_solve(L, U, e)
    
    return A_inv

print("Обратная матрица:")
A_inv_true =  np.array([[2/9., -1/3., 1/9.], [14/9., -1/3., -2/9.], [-7/9., 2/3., 1/9.]])
display(sp.Matrix(A_inv_true))

print("\nОбратная матрица, найденная с помощью LU-разложения:")
A_inv_lu = lu_inv(A)
display(sp.Matrix(A_inv_lu))
print(f"### Точность метода: {np.linalg.norm(A_inv_lu - A_inv_true, ord = 1)}")

print("\nОбратная матрица, найденная с помощью numpy.linalg.inv:")
A_inv_np = np.linalg.inv(A)
display(sp.Matrix(A_inv_np))
print(f"### Точность метода: {np.linalg.norm(A_inv_np - A_inv_true, ord = 1)}")


Обратная матрица:


Matrix([
[ 0.222222222222222, -0.333333333333333,  0.111111111111111],
[  1.55555555555556, -0.333333333333333, -0.222222222222222],
[-0.777777777777778,  0.666666666666667,  0.111111111111111]])


Обратная матрица, найденная с помощью LU-разложения:


Matrix([
[ 0.222222222222222, -0.333333333333333,  0.111111111111111],
[  1.55555555555556, -0.333333333333333, -0.222222222222222],
[-0.777777777777778,  0.666666666666667,  0.111111111111111]])

### Точность метода: 1.1102230246251565e-16

Обратная матрица, найденная с помощью numpy.linalg.inv:


Matrix([
[ 0.222222222222222, -0.333333333333333,  0.111111111111111],
[  1.55555555555556, -0.333333333333334, -0.222222222222222],
[-0.777777777777778,  0.666666666666667,  0.111111111111111]])

### Точность метода: 7.216449660063518e-16


## Часть 4. Модифицированный метод Гаусса

Модифицировать метод Гаусса из Части 1 так, чтобы система $A_1x = b_1$ решалась корректно. ВАЖНО: реализация метода должна быть получена путем модификации метода $\textit{gauss}$, а не переписыванием кода с нуля! 

In [636]:
def modified_gauss(A_in, b_in):
    n = b_in.size
    A = copy.deepcopy(A_in)
    b = copy.deepcopy(b_in)

    for k in range(0, n - 1):        
        row_max_element = np.argmax(A[:,k])
        A[[row_max_element, k]] = A[[k, row_max_element]]
        b[[row_max_element, k]] = b[[k, row_max_element]]

        for i in range(k + 1, n):
            if A[i,k] != 0:
                c = A[i,k]/A[k,k]
                A[i,(k):n] = A[i,(k):n] - c * A[k,(k):n]
                b[i] = b[i] - c * b[k]

    # Обратный ход
    for k in range(n - 1, -1, -1):
        b[k] = (b[k] - np.dot(A[k,(k + 1):n], b[(k + 1):n]))/A[k,k]
        
    return b

print('u1 = ', modified_gauss(A1, b1))
print('u2 = ', modified_gauss(A2, b2))

u1 =  [1. 1. 1.]
u2 =  [1. 1. 1.]
